#DNPV CODE 
#pip install numpy-financial
#pip install numpy

In [1]:
import os
import glob
import xlrd
import scipy.stats as st
import numpy as np
import numpy_financial as npf
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.dates import DateFormatter

In [2]:
def int_ab(Fx, xc, a, b, ns=1000):
  '''
  Calculates the area (Aab)and center of gravity (cg) by numerical integration from a to b  
  of a 1D function Fx with respect to xc where ns is the discretization parameter.
  Returns the area and center of gravity 
  
  :param Fx: F(x) function to integrate  
  :type a:float (lower limit)
  :type b:float (upper limit)
  :type xc:float reference point
  :type ns:int discritization parameter
  ''' 

  delta = (b - a)/ns
  x2= a  
  Aab = 0
  momX = 0
  for i in range(ns):
    x1 = x2 
    x2 = x1 + delta
    xm = (x1+x2)/2 
    dA = Fx(xm)*delta
    Aab = Aab + dA
    momX = momX + dA*(xm-xc)
  cg = momX/Aab
  return Aab, cg

In [3]:
def Risk_normal(p50=0, px = -3.0, x=0.9986, sig=1.0, sigf=True):
  '''
  Calculates the cost of risk of a random variable described by a normal probability density function (PDF).
  The PDF can be defined by:
   (1) the average (p50) and standard deviation (sig); or
   (2) p50, px and the associated cumulative probability x

  :type p50:float PDF average expected to be exceeded with 50% probability
  :type px:float represents the value expected to be exceeded with x% probability (default -3.0)
  :type x:float  represents the probability of exceedence (default 99.86%)
  :type sig:float standard deviation (default 1.0)
  :type sigf: bool indicates if the PDF will be assigned (False) or calculated from p50, px and x (True)
  '''
  if sigf:
    sig = (px-p50)/st.norm.ppf(1-x)

  #calculate the area of the downside and the center of gravity of the downside (cgd)
  #Because the PDF is symmetric, the distance from p50 to the cgd is the same for both sides of the curve 
  area, cgd = int_ab(st.norm.pdf,0,0,6)

  CostofRisk = cgd*sig*area
  if abs(p50)> 0: CostofRisk = CostofRisk/p50  
  return CostofRisk

In [4]:
def Risk_multinomial(p, x,ex = True, jf = 0, revenue=True):
  '''
  Calculates the cost of risk for a PDF defined by a multinomial distribution 
  p is a vector 
  x is a vector containing the corresponding events
  
  :type p: List containing the probabilities for each event x
  :type x: List containing the events
  ;type ex: bool flags if the base is the best (i.e., most likely) estimate (ex=False) or the average (ex=True)
  :type jf: indicates which event will be used as the based (default 0). Used when ex=False
  :type revenue: bool indicates is the PDF represents revenues or cost. Needed to evaluate the downside. 
  '''  
  ex_p = 0  
  risk = 0 
  j = len(p)
  for i in range(j):
    ex_p = ex_p + p[i]*x[i] 

  if ex: 
    base = ex_p  
  else: 
    base = x[jf]
    
  for i in range(j):
    if revenue: 
      risk = risk + p[i]*max(base-x[i],0) 
    else: 
      risk = risk + p[i]*max(x[i]-base,0) 

  if abs(base) > 0:
    costofrisk = risk/base
  else: costofrisk = risk
    
  return costofrisk

In [5]:
def getcashflowitems(xcelfile,srows=0, sheet_name ='DNPV Input', scols=0,fmt=''):
  '''
  Retrieves data from excel file found in a tab 'DNPV Input' and stores it in a 2D panda data structure. 
  Typical panda data structure have labels in the 1st row and data in columns
  To be used for data in xcelfile formatted in rows with id labes located in the 1st column. 
  Creates a dictionary with the excel data (df_dict) and a panda file (dfT) with labels in the 1st row 
  The first row of data could be located srows below
  '''  
  if fmt=='':
    fmt = '${:,.0f}'     

  pd.options.display.float_format = fmt.format
  
  df = pd.read_excel(xcelfile, sheet_name,skiprows=srows,header=None)
  nrows = len(df)
  ncolumns = len(df.columns) 
  df_dict = {}

  for i in range(nrows):
    values=[]
    keys = df[0][i]
    for j in range(1,ncolumns):
      values.append(df.iloc[i][j])
    df_dict[keys] = values
  dfT = pd.DataFrame(df_dict)

  return df_dict, dfT  
  

In [6]:
def riskparameters():
    #This is a placeholder for later implementation. 
    #At the moment, they are simply assigned and stored in a list call rp
    #Each risk should be interactively assigned via pulldown menus
    
    rp={}

    #Solar risk (revenue reduction)
    rp['Solar Risk']=['Revenues', True, Risk_normal(1164, 1088, 0.9)]
            
    #Maintence Risk (cost increase)
    p=[0.8,0.2]
    x=[1.0,1.25]
    rp['Maintenance Risk']=['Maintenance', True, Risk_multinomial(p,x, ex = False,revenue=False)]

    #Political Risk: FiT reduction (revenue reduction)
    p=[0.865,0.12,0.015]
    x=[0.43,0.28,0.06]
    rp['Political Risk']=['Revenues', False, Risk_multinomial(p,x, ex = False)]
 
    return rp



In [7]:
class Risk:
    
  def __init__(self, label, vector, fctr=1, keyparameters={}, time = 'Year', d=0):
    self.dict ={}                 #initialize dictionario of risks
    self.l = len(vector)          #vector length
    self.d = d                    #rounding decimal for print
    self.keyp = keyparameters     #dictionary containing all risk parameters used
    
    period = [i+1 for i in range(len(vector))]
    self.dict[time] = period
    
    if fctr!=1.0:
      vector = [x*fctr for x in vector]
    self.dict[label] = vector

  def assign(self,label,vector,fctr=1):
    if fctr!=1.0:
      vector = [x*fctr for x in vector]
    self.dict[label] = vector

  def add(self, label='', keys=[]):
    keyparam = list(self.keyp.keys())
    arrayt=np.array([0 for x in range(self.l)])
        
    for key in keyparam: 
      arrayt = arrayt + np.array(self.dict[key])

    self.dict[label] = arrayt.tolist()

  def sub(self, label, a, b, fa = 1, fb = 1):
    array_c = np.array(a)*fa - np.array(b)*fb
    self.dict[label] = array_c.tolist()
   
  def __str__(self):
    keys = list(self.dict.keys())
    keys.pop(0) #removes the year list
    txt = '{'
    for key in keys:
        vector = self.dict[key] 
        prn = [round(val,self.d) for val in vector]
        txt = txt + f"'{key}'" +':'+ str(prn) + '\n\n'  
    txt = txt[:-3]+'}'    
        
    return txt



In [8]:
def DNPVelements(cashflow, riskparameters, sumkey='Sum Risk', rcfkey='DNPV-CF'):
  global rate_f, tax
  risk = None
 
  for key in list(riskparameters.keys()):
        cfitem, tind, nriskcost = riskparameters[key]
        vector = cashflow[cfitem]

        if tind == False:    
            dim = len(vector)  
            tvector=[]
            for i in range(dim):
              sum = 0
              for j in range(i,dim):
                sum = sum + vector[j]/pow(1+rate_f+nriskcost,j-i+1)
              tvector.append(sum*nriskcost)
            vector = tvector
            nriskcost = 1

        if risk == None: 
            risk = Risk(key,vector, nriskcost, d=1)
            risk.keyp = riskparameters
        else: 
            risk.assign(key, vector, nriskcost)

  #Add all the calculated cost of risks and assign it to risk.dict[sumkey]  
  risk.add(sumkey)

  #calculate the riskless cashflows and assign it to risk.dict[k2]  
  key0 = list(cashflow.keys())[-1]
  risk.sub(rcfkey, cashflow[key0],risk.dict[sumkey],fb = 1-tax)
    
  return risk, rcfkey
    

In [10]:
    #Obtain the items needed to estimate the cost of risks
    #Excel tab should have the rows with data affecred by risk
    #All the data below should be provided 
    
    tax = 0.333     #corporate tax rate
    rate_f = 0.028  #risk-free rate
    wacc = 0.13     #investor's capital cost

    CFfilename = 'Solar Project 2024.xlsx'
    rowloc = 10

    #get cashflow items in a dictionary and panda dataframe formats
    cashflow, dfT = getcashflowitems(CFfilename,rowloc)

    #get the risk parameters in a dictionary format to calculate DNPV
    rp = riskparameters()

    #get the cost of risk line items and calculate the DNPV riskless cash flows
    risk, keydnpv = DNPVelements(cashflow, rp)    

    dnpvvec = risk.dict[keydnpv]
    
    init = [0]  # to be included as period 0 if teh cashflows do not include t=0 data
    
    dnpv = npf.npv(rate_f,init+dnpvvec)

    keycf = list(cashflow.keys())[-1]
    npv  = npf.npv(wacc,init+cashflow[keycf])

    print(f"DNPV: ${dnpv:,.0f}" )   

    risk_pdmatrix = pd.DataFrame(risk.dict)
    risk_pdmatrix
    

DNPV: $15,265,231


,Year,Solar Risk,Maintenance Risk,Political Risk,Sum Risk,DNPV-CF
0,1,"$40,729","$6,253","$1,227,011","$1,273,992","$616,006"
1,2,"$41,580","$6,409","$1,218,822","$1,266,811","$642,061"
2,3,"$42,449","$6,569","$1,207,662","$1,256,680","$670,445"
3,4,"$43,336","$6,733","$1,193,236","$1,243,305","$701,357"
4,5,"$44,242","$6,902","$1,175,226","$1,226,369","$735,014"
5,6,"$45,166","$7,074","$1,153,284","$1,205,525","$771,650"
6,7,"$46,110","$7,251","$1,127,035","$1,180,396","$811,520"
7,8,"$47,074","$7,432","$1,096,070","$1,150,576","$854,899"
8,9,"$48,058","$7,618","$1,059,945","$1,115,621","$902,087"
9,10,"$49,062","$7,809","$1,018,179","$1,075,050","$953,406"
